In [3]:
!pip install transformers
!pip install telebot




Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 545.4/545.4 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.8 MB/s eta 0:00:00


In [21]:

!pip install --upgrade python-telegram-bot



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [22]:
import torch
from transformers import AutoTokenizer, BertForSequenceClassification
from telegram import Update
from telegram.ext import Updater, CommandHandler, MessageHandler, CallbackContext
from telegram.ext.message_filters import filters

# Define the Telegram bot token
TOKEN = "your_token_here"

# Load pre-trained BERT model and tokenizer
model_path = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_path)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define hyperparameters
learning_rate = 1e-5
num_epochs = 3
batch_size = 64
max_len = 128

# Load and preprocess the data
def preprocess_data():
    # Implement your data loading and preprocessing logic here
    # Return tokenized datasets or any required data structures
    pass

# Train the model
def train_model(tokenized_datasets):
    classifier = BertForSequenceClassification.from_pretrained(model_path, num_labels=3).to(device)
    optimizer = torch.optim.Adam(classifier.parameters(), lr=learning_rate)

    for epoch in range(num_epochs):
        for inputs, labels in tokenized_datasets:
            inputs = tokenizer.encode_plus(inputs, max_length=max_len, padding="max_length", truncation=True, return_tensors="pt")
            inputs = inputs.to(device)
            labels = torch.tensor(labels).to(device)

            optimizer.zero_grad()
            outputs = classifier(**inputs, labels=labels)
            loss = outputs.loss
            loss.backward()
            optimizer.step()

# Telegram Bot handlers
def start(update: Update, context: CallbackContext):
    update.message.reply_text("Hello! Send me a text and I will classify it using BERT.")

def classify_text(update: Update, context: CallbackContext):
    text = update.message.text
    inputs = tokenizer.encode_plus(text, max_length=max_len, padding="max_length", truncation=True, return_tensors="pt")
    inputs = inputs.to(device)

    classifier = BertForSequenceClassification.from_pretrained(model_path, num_labels=3).to(device)
    classifier.eval()

    with torch.no_grad():
        outputs = classifier(**inputs)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=-1)

    prediction = preds.item()
    update.message.reply_text(f"Predicted class: {prediction}")

# Initialize and start the bot
def initialize_bot():
    tokenized_datasets = preprocess_data()
    train_model(tokenized_datasets)

    updater = Updater(TOKEN, use_context=True)
    dp = updater.dispatcher

    dp.add_handler(CommandHandler("start", start))
    dp.add_handler(MessageHandler(filters.text & ~filters.command, classify_text))

    updater.start_polling()
    updater.idle()

# Call the function to initialize and start the bot
initialize_bot()


ModuleNotFoundError: ignored